In [2]:
# Module
import numpy as np

def sigmoid(x):
    return 1.0/(1.0+np.exp(-x))

print(sigmoid(-11)) 

1.670142184809518e-05


In [1]:
%run Single_Layer_Percept_전복고리수.ipynb

Epoch 1 : loss = 33.906, /t accuracy =0.557/0.801
Epoch 2 : loss = 8.225, /t accuracy =0.820/0.755
Epoch 3 : loss = 7.582, /t accuracy =0.812/0.865
Epoch 4 : loss = 7.476, /t accuracy =0.808/0.785
Epoch 5 : loss = 7.397, /t accuracy =0.810/0.820
Epoch 6 : loss = 7.331, /t accuracy =0.808/0.779
Epoch 7 : loss = 7.272, /t accuracy =0.808/0.810
Epoch 8 : loss = 7.220, /t accuracy =0.808/0.838
Epoch 9 : loss = 7.178, /t accuracy =0.810/0.838
Epoch 10 : loss = 7.138, /t accuracy =0.809/0.860

filnal Test : final accuracy = 0.860
[[1.03338961]
 [1.47724746]
 [1.67992711]
 [2.03717282]
 [1.60645146]
 [0.62703825]
 [2.42585997]
 [0.52521222]
 [0.46959183]
 [0.96029006]] [4.16916919]
Epoch 5 : loss = 6.807, /t accuracy =0.809/0.808
Epoch 10 : loss = 6.474, /t accuracy =0.813/0.794
Epoch 15 : loss = 6.237, /t accuracy =0.818/0.818
Epoch 20 : loss = 6.018, /t accuracy =0.821/0.811
Epoch 25 : loss = 5.843, /t accuracy =0.824/0.805
Epoch 30 : loss = 5.696, /t accuracy =0.826/0.811
Epoch 35 : loss =

In [2]:
# 메인함수 정의
def pulsar_exec(epoch_count=10,mb_size=10,report=1):
    load_pulsar_dataset()
    init_model()
    train_test(epoch_count,mb_size,report)

In [3]:
# 데이터 적재함수 정의
def load_pulsar_dataset():
    with open("pulsar_data_train.csv") as csvfile:
        csvreader=csv.reader(csvfile)
        next(csvreader,None)
        rows=[]

        for row in csvreader:
            for i in range(0,len(row)) :
                if row[i]=='':row[i]=0
            rows.append(row)

    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 8,1
    data = np.asarray(rows,dtype=np.float32)


In [4]:
#  후처리 과정에 대한 순전파와 역전파 함수 재정의
def forward_postproc(output,y):
    entropy = sigmoid_cross_entropy_with_logits(y,output)
    loss=np.mean(entropy)
    return loss,[y,output,entropy]

def backprop_postproc(G_loss,aux):
    y,output,entropy=aux
    
    g_loss_entropy=1.0/np.prod(entropy.shape)
    g_entropy_output=sigmoid_cross_entropy_with_logits_derv(y,output)
    
    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy
    
    return G_output

In [5]:
# 정확도 함수 재정의
def eval_accuracy(output,y):
    estimate = np.greater(output,0)
    answer=np.greater(y,0.5)
    correct = np.equal(estimate,answer)
    
    return np.mean(correct)

In [6]:
# 시그모이드 관련 함수
def relu(x):
    return np.maximum(x,0)

def sigmoid(x):
    return np.exp(-relu(-x))/ (1 + np.exp(-np.abs(x)))

def sigmoid_derv(x,y):
    return y*(1-y)

def sigmoid_cross_entropy_with_logits(z,x):
    return relu(x)-x*z+np.log(1+np.exp(-np.abs(x)))

def sigmoid_cross_entorpy_with_logits_derv(z,x):
    
    return -z + sigmoid(x)

In [7]:
pulsar_exec()

NameError: name 'sigmoid_cross_entropy_with_logits_derv' is not defined

In [ ]:
'''
그러나 data set 에서 일반 별은 90%, 펄서의 비율 9.2%이기 때문에 모두 일반 별이라 대답해도 정확도는 90%.
따라서 정확한 학습을 위해 일반별과 펄서의 빈도를 조정할 필요가 있음.
일반별의 데이터를 펄서와 같은 수로 줄이는 것보다, 일반 별의 데이터만큼 펄서 데이터를 중복 시켜 수를 늘리는 것이 좋다.

또한, 퍼포먼스 지표도 수정이 필요하다.
데이터 균형을 맞춤으로써 신경망의 추정 성능이 향상되지만, 정확도는 하락할 수 있기 때문이다.
이러한 이유로 정확도 대신 신경망의 성능 측정에 더 적합한 평가 지표가 요구됨.

퍼포먼스 측정 지표의 종류
1) 정밀도 : 참으로 추정한 것 가운데 정답이 참인 것의 비율
2) 재현율 : 정답이 참인 것들 가운데 신경망이 참으로 추정한 것의 비율

>> 2개 지표를 합친 F1 지표(정밀도와 재현율의 조화평균)
F1 = (2*재현율*정밀도)/ (재현율+정밀도)
'''

'''
아래 코드에서는 펄서 1) 데이터를 중복 사용하고 2) 이 때 성능을 평가지표로 확인
'''

In [8]:
# 메인함수 재정의
def pulsar_exec(epoch_count=10,mb_size=10,report=1,adjust_ratio=False):
    load_pulsar_dataset(adjust_ratio)
    init_model()
    train_test(epoch_count,mb_size,report)

In [9]:
# 데이터 적재함수 정의
def load_pulsar_dataset(adjust_ratio):
    pulsars, stars =[],[]
    with open("pulsar_data_train.csv") as csvfile:
        csvreader=csv.reader(csvfile)
        next(csvreader,None)
        rows=[]

        for row in csvreader:
            for i in range(0,len(row)) :
                if row[i]=='':row[i]='0.0'
            if row[8]=='1.0': pulsars.append(row)
            else: stars.append(row)

                
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 8,1
    
    star_cnt,pulsar_cnt = len(stars),len(pulsars)
  
    if adjust_ratio:
        data = np.zeros([2*star_cnt,9])
        data[0:star_cnt,:]=np.asarray(stars,dtype='float32')# adjust_ratio 가 True 이면 별데이터를 data 버퍼에 담은 후 
        for n in range(star_cnt):    # 반복문으로 펄서 데이터를 별 데이터와 같은 수만큼 data 버퍼에 담음
            data[star_cnt+n] = np.asarray(pulsars[n % pulsar_cnt],dtype='float32')
            
    else:  # 별과 펄서 데이터 차례로 data 버퍼에 담기
        data = np.zeros([star_cnt+pulsar_cnt,9])
        data[0:star_cnt,:]=np.asarray(stars,dtype='float32')
        data[star_cnt,:]=np.asarray(pulsars,dtype='float32')

In [10]:
# 정확도 계산 함수 재정의
def eval_accuracy(output,y):
    est_yes = np.greater(output,0)
    ans_yes = np.greater(y,0.5)
    est_no = np.logical_not(est_yes)
    ans_no = np.logical_not(ans_yes)
    
    tp = np.sum(np.logical_and(est_yes,ans_yes))
    fp = np.sum(np.logical_and(est_yes,ans_no))
    tn = np.sum(np.logical_and(est_no,ans_no))
    fn = np.sum(np.logical_and(est_no,ans_yes))

    accuracy =safe_div(tp+tn,tp+tn+tn+fn)
    precision=safe_div(tp,tp+fp)
    recall=safe_div(tp,tn+fn)
    f1=2*safe_div(recall*precision,recall+precision)
    
    return [accuracy,precision,recall,f1]


def safe_div(p,q):  # 타입 오류 방지를 위한 형 변환, 0 나누기 오류 방지를 위한 예외처리
    p,q=float(p),float(q)
    if np.abs(q) < 1.0e-20 : return np.sign(p)

    return p/q    

In [11]:
# train & test set
def train_test(epoch_count,mb_size,report):
    step_count=arrange_data(mb_size)  # 데이터 셔플, train & test 셋 분리, 정렬 작업 수행
    test_x,test_y=get_test_data()
    
    for epoch in range(epoch_count):  # 인수로 지정된 에포크 수만큼 학습 반복
        losses=[]
        
        for n in range(step_count):  # step_count 값만큼 미니배치 처리 반복
            train_x,train_y =get_train_data(mb_size,n)  
            loss, _ = run_train(train_x,train_y)   # train 데이터 가져와 학습
            losses.append(loss)     # 미니배치에서 손실과 정확도 값을 각각 적재
                
        if report > 0 and (epoch+1)  % report ==0:  # report : 보고주기. 보고주기에 맞으면 손실 및 정확도를 출력
            acc=run_test(train_x,train_y )
            print('Epoch {} : loss = {:5.3f}, /t accuracy ={:5.3f}/{:5.3f}'.format(epoch+1,np.mean(losses),acc_str))
    
    acc =run_test(train_x,train_y )  # 최종 테스트 함수 호출 및 값 출력
    acc_str=",".join(["%5.3f"]*4)%tuple(acc)
    print('\nFilnal Test : final result = {}'.format(acc_str))


In [12]:
pulsar_exec()

ValueError: could not broadcast input array from shape (1153,9) into shape (9)

In [13]:
with open("pulsar_data_train.csv") as csvfile:
    csvreader=csv.reader(csvfile)
    next(csvreader,None)
    rows=[]

    for row in csvreader:
        for i in range(0,len(row)) :
            if row[i]=='':row[i]='0'
                
    for row in csvreader:
        if row[8]=='1.0': pulsars.append(row)
        else: stars.append(row)

data = np.zeros([star_cnt+pulsar_cnt,9])
data[0:star_cnt,:]=np.asarray(stars,dtype='float32')
# data[star_cnt,:]=np.asarray(pulsars,dtype='float32')

NameError: name 'star_cnt' is not defined

NameError: name 'star' is not defined

9